In [1]:
import ipywidgets as widgets
import numpy as np
import keras

from IPython import display as nb
import utils

2024-12-11 21:55:46.064197: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-11 21:55:46.137261: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-11 21:55:46.158687: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-11 21:55:46.212109: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-11 21:55:47.383100: W tensorflow/compiler/tf2

In [2]:
packed_model = keras.models.load_model("../model/2024-12-11_21-53-19.keras")
packed_model.layers

I0000 00:00:1733928949.464238    1292 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1733928949.534635    1292 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1733928949.534712    1292 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1733928949.540760    1292 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1733928949.540848    1292 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:0

[<WordPieceTokenizer name=word_piece_tokenizer, built=False>,
 <StringLookup name=string_lookup, built=False>,
 <Sequential name=sequential, built=True>]

In [3]:
tokenizer = packed_model.layers[0]
label_lookup = packed_model.layers[1]

model = packed_model.layers[2]
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 74, 64)         │        33,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 256)            │       197,632 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 32)             │         8,224 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 4)              │           132 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 718,958 (2.74 MB)

 Trainable params: 239,652 (936.14 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 479,306 (1.83 MB)

In [ ]:
w_text = widgets.Text(placeholder="Task")
w_text.continuous_update = False
nb.display(w_text)

def on_submit(value):
    nb.clear_output()
    nb.display(w_text)

    task = value["new"].strip()
    print(f"Input: {task}")

    task = utils.lemmatize(task)
    print(f"Lemmatized input: {task}")

    task_vec = tokenizer(task).numpy()
    print(f"Vectorized input: {tokenizer.detokenize(task_vec)}")
    print(f"{task_vec}\n")
    task_vec = np.reshape(task_vec, (1, len(task_vec)))

    prediction = model.predict(task_vec)[0]
    print(f"Prediction vector: {prediction}")

    label_vocab = label_lookup.get_vocabulary()
    for label, probability in zip(label_vocab, prediction):
        percentage = probability * 100
        print(f"\t{label}: {percentage:.5f}%")

    best = label_vocab[np.argmax(prediction)]
    print(f"Prediction: {best}")

w_text.observe(on_submit, "value")


Text(value='finish progress report', continuous_update=False, placeholder='Task')

Input: finish progress report
Lemmatized input: finish progress report
Vectorized input: finish progress report [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
[365 285  75   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
Prediction vector: [0.00240994 0.03339249 0.68104696 0.28315067]
	not-urgent not-important: 0.24099%
	not-urgent importan

In [ ]:
model.input_shape

(None, 74)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step